# Deliverable 1

In [76]:
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder

Import the csv file with data from ABP

In [77]:
filename = "data/abp_data.csv"

df = pd.read_csv(filename)
df.columns

Index(['permitnumber', 'worktype', 'permittypedescr', 'description',
       'comments', 'applicant', 'declared_valuation', 'total_fees',
       'issued_date', 'expiration_date', 'status', 'owner', 'occupancytype',
       'sq_feet', 'address', 'city', 'state', 'zip', 'property_id',
       'parcel_id', 'lat', 'long'],
      dtype='object')

## Preprocessing

We decided for our first pass at the data to simply get the frequency of each applicant and each owner in the dataset. 

This will show us which individuals are applying for housing permits on behalf of an org the most and which orgs are requesting permits the most.

In [78]:
applicants = pd.DataFrame(df['applicant'])
owners = pd.DataFrame(df['owner'])
applicants

,applicant
0,Renee Santeusanio
1,Jusimar Oliveria
2,Andreas Hwang
3,NaN
4,Ping Mandawe
...,...
478014,Christine McMahon
478015,Christine McMahon
478016,Regina Olivieri
478017,Maria Dubrowski


Here we fill all the missing entries with the string "na" and cast all the entries to strings

In [79]:
applicants.fillna("na")
applicants = applicants.applymap(str)

owners.fillna("na")
owners = owners.applymap(str)

Here we define a function for prepping a column to be encoded. To eliminate inconsistencies across how the names were entered, we make all the entries lowercase and remove the spaces.

In [80]:
def prep_df_column(col, generate_map=False):
    """
    col: A pandas column-frame of strings
    generate_map: Whether or not to generate a map of the original values to the edited values 

    Outputs the columm with all the entries set to lowercase and with spaces removed
    """
    new_col = col.iloc[:, 0]
    new_col = [name.lower().replace(" ", "") for name in new_col]
    col_map = dict()
    if generate_map:
        for i in range(len(new_col)):
            col_map[new_col[i]] = col.iloc[:, 0][i]
        return new_col, col_map
    else:
        return new_col

We utilize the label encoder that we saw in class to encode the different string values in the columns, since we just need to get a count of the distinct entries. It probably isn't quite necessary here but it will be useful for future processing to get a handle on it.

In [81]:
# we split the encoders into two so they can keep track of their inverse transforms
applcants_encoder = LabelEncoder()
owners_encoder = LabelEncoder()

# here we also optionally output the mapping between the preprocessed values and the original, which we can use for display purposes
# however, building the map takes longer than we'd like
# applicants_cleaned, applicants_map = prep_df_column(applicants, generate_map=True)
# owners_cleaned, owners_map = prep_df_column(owners, generate_map=True)

applicants_cleaned = prep_df_column(applicants, generate_map=False)
owners_cleaned = prep_df_column(owners, generate_map=False)

applicants_labels = pd.DataFrame(applcants_encoder.fit_transform(applicants_cleaned))
owners_labels = pd.DataFrame(owners_encoder.fit_transform(owners_cleaned))

Here we show the names of the original strings with the encoded values for display purposes.

In [82]:
applicants_labels_with_names = applicants_labels.copy()
applicants_labels_with_names['names'] = applicants

applicants_labels_with_names

,0,names
0,30653,Renee Santeusanio
1,20069,Jusimar Oliveria
2,1256,Andreas Hwang
3,27060,nan
4,30052,Ping Mandawe
...,...,...
478014,5505,Christine McMahon
478015,5505,Christine McMahon
478016,30604,Regina Olivieri
478017,23488,Maria Dubrowski


In [83]:
owners_labels_with_names = owners_labels.copy()
owners_labels_with_names['names'] = owners

owners_labels_with_names

,0,names
0,12221,CITY OF BOSTON
1,48278,RUBIO FAMILY TRUST LLC
2,32287,LEDERMAN US REAL ESTATE CORP
3,37460,MIARA SIMON
4,34217,MABB LLC
...,...,...
478014,50819,SHIGO CENTER PLAZA OWNER LLC
478015,50819,SHIGO CENTER PLAZA OWNER LLC
478016,5524,AVONWOOD ASSOCS LPS
478017,21276,FORTY 6 BURROUGHS ST CONDO


## Data Synthesis

Now that the data has been preprocessed, we will start by getting the counts of all the distinct applicants and owners.

Here we can see Robert Trethewey has been applying on be half of the most distinct owners.

In [84]:
applicants_labels_vals = applicants_labels.value_counts()
index = [tup[0] for tup in applicants_labels_vals.index]
applicants_labels_vals.index = applcants_encoder.inverse_transform(index)
pd.DataFrame(applicants_labels_vals)

,0
nan,14215
roberttrethewey,5892
johnmclaughlin,3603
paulguarracino,3253
richardfallone,3059
...,...
kevinlaik,1
kevinlang,1
kevinlarnach,1
kevinlau,1


And here we can see that the city of Boston has applied for the most permits.

In [85]:
owners_labels_vals = owners_labels.value_counts()
owners_index = [tup[0] for tup in owners_labels_vals.index]
owners_labels_vals.index = owners_encoder.inverse_transform(owners_index)
pd.DataFrame(owners_labels_vals)

,0
nan,36903
cityofboston,6003
bostonhousingauthority,2633
marriottownershipresorts,2546
northeasternuniversity,2523
...,...
leachdaphney,1
twenty1marylandst,1
leabokarld,1
le-royangusjetal,1


Here we take a look at the list of orgs that the top applicant has applied for. Our search is somewhat incomplete as we don't search on the preprocessed value, but it gives a good sense of the amount of orgs.

In [86]:
df1 = df[applicants['applicant'].str.contains("Robert Trethewey")]
df1.drop_duplicates("owner")["owner"]

67609                  OCONNOR CAROL A
188501             EGAN PATRICK ROBERT
189588                  ALJOE NICOLE N
204901        SEVENTY 2-74 EAST DEDHAM
204954    WEDIKO CHILDRENS SERVICES IN
                      ...             
340714                 HORENSTEIN MARK
344617          BELGRADE AND BIRCH LLC
345559               ENGLAND ALEXANDRA
345754            SIXTEEN GLADE AVENUE
429745        SEVENTEEN OAKDALE STREET
Name: owner, Length: 775, dtype: object